## GET RID OF SPACES IN FRAME PARCEL NUMBER

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

In [2]:
#parcel files very large, stored outside of project folder in github root folder
root_path = os.getcwd().split('/phx')[0]
data_dir = '/data_large/'
parcels_shape, parcels_tax = 'parcels_shape','parcels_tax'
files = ['100','200','300','400','500']

In [3]:
tax_column_headers = ['PARCEL_NUMBER','OWNER_NAME','OWNER_MAIL_ADDRESS_1','OWNER_MAIL_ADDRESS_2',\
                  'OWNER_MAIL_CITY','OWNER_MAIL_STATE','OWNER_MAIL_ZIP_CODE','OWNER_MAIL_COUNTRY','SITUS_ADDRESS',\
                  'SITUS_SUITE','SITUS_CITY','SITUS_ZIP_CODE','PROPERTY_TYPE','DEED_NUMBER','DEED_DATE',\
                  'DEED_TYPE','LAND_FULL_CASH_VALUE','IMPROVEMENT_FULL_CASH_VALUE',\
                  'TOTAL_FULL_CASH_VALUE','TOTAL_FULL_CASH_ASSESSED_VALUE','LIMITED_PROPERTY_VALUE',\
                  'LPV_ASSESSED','PROPERTY_USE_CODE','LOT','BLOCK','TRACT','LAND_LEGAL_CLASS_CODE',\
                  'LAND_LEGAL_CLASS_SUB_CODE','IMPROVEMENT_LEGAL_CLASS_CODE','IMPROVEMENT_LEGAL_CLASS_SUB_CODE',\
                  'RENTAL_INDICATOR','LIVING_AREA','TOTAL_SQ_FT','CONSTRUCTION_YEAR','POOL_SIZE','SALE_PRICE',\
                  'SALE_DATE','MCR_NUMBER','SUBDIVISION_NAME','NUMBER_OF_UNITS','TAX_AREA_CD']

In [5]:
#parcel data from City Open Data
city = gpd.read_file(f'{root_path}/{data_dir}{parcels_shape}/City_Parcels.shp')
city['APN_2'] = city['APN'].str.replace('-','')
city_APN_list = list(city.APN_2.unique().astype(str))

In [6]:
#parcel data from city txt files
frame = pd.DataFrame()
for file in files:
    df = pd.read_table(f'{root_path}/{data_dir}{parcels_tax}/R115-BK{file}_2024.TXT',
                       delimiter='|',
                       header=None,
                      low_memory=False)
    frame = pd.concat([frame,df])
    
frame.columns = tax_column_headers

In [7]:
#clean up the parcel ids
frame.PARCEL_NUMBER = frame.PARCEL_NUMBER.str.replace(' ','',regex=False)

### read in building permit data

In [8]:
permits = pd.read_csv('../data/kiva_cofc_23.csv')

In [9]:
permit_apns = [*set(list(permits.APN_2))]

### join tax data to APNs 

In [10]:
perm2 = pd.merge(permits,frame,how='left',left_on='APN_2',right_on='PARCEL_NUMBER')

In [20]:
perm3.head(20)

,Year,PARENT_PERMIT_TYPE,PER_TYPE_DESC,PARENT_PERMIT_NUMBER,PARENT_PERMIT_STATUS,PARENT_PERMIT_ISSUE_DATE,PROJECT_NUMBER,PROJECT_DESCRIPTION,PERMIT_NAME,ADDRESS,...,LIVING_AREA,TOTAL_SQ_FT,CONSTRUCTION_YEAR,POOL_SIZE,SALE_PRICE,SALE_DATE,MCR_NUMBER,SUBDIVISION_NAME,NUMBER_OF_UNITS,TAX_AREA_CD
0,2010,RSF,RES SINGLE FAMILY,9018057,DONE,40101.00000,99-40955,ROGERS RANCH,LOT 12 STANDARD RESIDENCE,4637 W CARSON RD,...,1687,5500,2010.0,,,042022,78142,ROGERS RANCH PARCEL 8 ...,0.0,591300.0
1,2010,RSF,RES SINGLE FAMILY,9014444,DONE,40039.63602,99-1525,DYNAMITE MOUNTAIN RANCH,LOT 75 STANDARD RESIDENCE,27915 N 17TH AVE,...,2884,11598,2009.0,432,524000,062019,79606,DYNAMITE MOUNTAIN RANCH PARCEL 31-A1/A2 ...,0.0,971300.0
2,2010,RSF,RES SINGLE FAMILY,7034344,DONE,39395.00000,01-20732,ORANGE GROVES ESTATES,NEW CUSTOM RES,5112 N 36TH ST,...,4386,9958,2008.0,400,1114000,072016,17037,ORANGE GROVE ESTATES ...,0.0,141300.0
3,2010,RSF,RES SINGLE FAMILY,9017482,DONE,40092.00000,02-1682,STETSON VALLEY PHASE II/III,LOT 188 STANDARD RESIDENCE,27506 N 54TH LN,...,1398,6125,2009.0,,225000,082014,78947,STETSON VALLEY PARCELS 30 31 32 33 ...,0.0,971300.0
4,2010,RSF,RES SINGLE FAMILY,9016368,DONE,40072.00000,02-1682,STETSON VALLEY PHASE II/III,LOT 187 STANDARD RESIDENCE,27510 N 54TH LN,...,2103,6125,2009.0,,,022010,78947,STETSON VALLEY PARCELS 30 31 32 33 ...,0.0,971300.0
5,2010,RSF,RES SINGLE FAMILY,9014826,DONE,40046.00000,03-1569,LAVEEN COMMONS,LOT 115 STANDARD RESIDENCE,8809 S 53RD DR,...,2534,11200,2010.0,432,255970,012010,75334,LAVEEN COMMONS ...,0.0,591300.0
6,2010,RSF,RES SINGLE FAMILY,9018656,DONE,40112.42108,99-38146,COUNTRY PLACE MASTER PLAN COMMUNITY,LOT 75 STANDARD RESIDENCE,10426 W HUGHES DR,...,1843,4950,2010.0,,240000,022019,87712,COUNTRY PLACE PHASE 4 PARCELS 11 12 AMD ...,0.0,651300.0
7,2010,RSF,RES SINGLE FAMILY,9018658,DONE,40112.42206,99-38146,COUNTRY PLACE MASTER PLAN COMMUNITY,LOT 76 STANDARD RESIDENCE,10430 W HUGHES DR,...,1444,4950,2010.0,,205000,082018,87712,COUNTRY PLACE PHASE 4 PARCELS 11 12 AMD ...,0.0,651300.0
8,2010,RSF,RES SINGLE FAMILY,9017064,DONE,40084.00000,00-5280,FARMINGTON GLEN,LOT 514 - STANDARD RESIDENCE,10128 W CHIPMAN RD,...,1400,5625,2010.0,,230000,062020,69716,FARMINGTON GLEN ...,0.0,651300.0
9,2010,DEM,DEMOLITION,9019858,DONE,40129.00000,06-1022,BAYLESS INVESTMENTS,TOTAL DEMOLITION,4110 N CENTRAL AVE,...,,53579,0.0,,17900000,122019,,...,0.0,81300.0


In [13]:
perm3 = perm2.copy().drop(['CHILD_PERMIT_TYPE','CHILD_PERMIT_NUMBER','CHILD_PERMIT_ISSUE_DATE','CHILD_PERMIT_STATUS'],axis=1)

In [16]:
perm3.shape

(39649, 66)

In [15]:
perm3 = perm3.drop_duplicates()

In [21]:
perm3.PARENT_PERMIT_TYPE.unique()

array(['RSF', 'DEM', 'RDEM', 'BLD', 'MHZ', 'RSME', 'INSP', 'BLDS', 'RSFC',
       'RFEN', 'RS', 'CCO', 'CDW', 'BLDA', 'FBBR', 'OE', 'COFC', 'RSC',
       'BLSC', 'RPBI', 'RSE', 'RSFA', 'RE', 'TCO', 'OP', 'OS', 'RM',
       'FEN', 'OBLD', 'F805'], dtype=object)

In [22]:
perm3.PER_TYPE_DESC.unique()

array(['RES SINGLE FAMILY', 'DEMOLITION', 'RES DEMOLITION',
       'STRUC/ELEC/PLMB/MECH', 'MOBILE HOME PLACEMENT-ZONING',
       'RES STRUC/MECH OR PLMB/ELEC',
       'COMMERCIAL INSPECTIONS ONLY PERMIT',
       'SHELL - STRUC/ELEC/PLMB/MECH',
       'RESIDENTIAL SINGLE FAMILY - SELF CERT', 'RES FENCE - ZONING',
       'RES STRUC', 'CONCRETE CIVIL PERMIT', 'DRIVEWAY CIVIL PERMIT',
       'STRUC/ELEC/PLMB/MECH - ADAPTIVE REUSE',
       'FACTORY BUILT BUILDING RES SALES OFF', 'OTC ELEC',
       'CERTIFICATE OF COMPLETION', 'RESIDENTIAL PERMIT - SELF CERT',
       'COMMERCIAL PERMIT - SELF CERT', 'RES PERMIT BY INSPECTION',
       'RES STRUC/ELEC', 'RES SINGLE FAMILY ATTACHED', 'RES ELEC',
       'TEMPORARY CERTIFICATE OF OCCUPANCY', 'OTC PLMB', 'OTC STRUC',
       'RES MECH OR PLMB', 'FENCE - ZONING ONLY',
       'OTC STRUC/ELEC/PLMB/MECH', 'FP FIRE ALARM MOD-SPEC EGRESS-OTC'],
      dtype=object)

In [23]:
perm3.to_excel('perm_explore.xlsx',index=False)